# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 

Permission denied: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
Maybe you need to change permission over 'Anyone with the link'?
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.
drive  model.ckpt  sample_data


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [6]:
import numpy as np

print('Loading data ...')

data_root='./drive/MyDrive/Colab Notebooks/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [7]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [8]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [9]:
BATCH_SIZE = 48
print("Batch Size : ", BATCH_SIZE)

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Batch Size :  48


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [10]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

282

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [21]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(9900, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 256)
        self.layer4 = nn.Linear(256, 128)
        self.layer5 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 39)
        self.act_fn1 = nn.Sequential(
          nn.ReLU(),
          nn.ReLU(),
          nn.ReLU(),
          nn.ReLU(),
          nn.ReLU()
        )
        self.BatchNorm1 = nn.BatchNorm1d(num_features=1024, eps=1e-5)
        self.BatchNorm2 = nn.BatchNorm1d(num_features=512, eps=1e-5)
        self.BatchNorm3 = nn.BatchNorm1d(num_features=256, eps=1e-5)
        self.BatchNorm4 = nn.BatchNorm1d(num_features=128, eps=1e-5)
        self.BatchNorm5 = nn.BatchNorm1d(num_features=64, eps=1e-5)
        self.Dropout = nn.Dropout(p = 0.3)

        self.cnn1 = nn.Conv1d(in_channels = 11, out_channels = 300, kernel_size = 7)
      
    def forward(self, x):
        x = torch.reshape(x, (inputs.shape[0], 11, 39))
        x = self.cnn1(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)
        x = torch.reshape(x, (-1, 9900))

        x = self.layer1(x)
        x = self.BatchNorm1(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)
        
        x = self.layer2(x)
        x = self.BatchNorm2(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)

        x = self.layer3(x)
        x = self.BatchNorm3(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)

        x = self.layer4(x)
        x = self.BatchNorm4(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)

        x = self.layer5(x)
        x = self.BatchNorm5(x)
        x = self.act_fn1(x)
        x = self.Dropout(x)

        x = self.out(x)
        return x

## Training

In [22]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [23]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [24]:
# fix random seed for reproducibility
same_seeds(891124)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 300               # number of training epoch
learning_rate = 1e-4          # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
count = 0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    count += 1

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                count = 0
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
    if count == 50:
      break

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/100] Train Acc: 0.519651 Loss: 1.708486 | Val Acc: 0.662881 loss: 1.126210
saving model with acc 0.663
[002/100] Train Acc: 0.606877 Loss: 1.355307 | Val Acc: 0.692244 loss: 1.013853
saving model with acc 0.692
[003/100] Train Acc: 0.636452 Loss: 1.250235 | Val Acc: 0.709542 loss: 0.949783
saving model with acc 0.710
[004/100] Train Acc: 0.654707 Loss: 1.184052 | Val Acc: 0.716729 loss: 0.918349
saving model with acc 0.717
[005/100] Train Acc: 0.669549 Loss: 1.131669 | Val Acc: 0.721457 loss: 0.894911
saving model with acc 0.721
[006/100] Train Acc: 0.681452 Loss: 1.089887 | Val Acc: 0.727571 loss: 0.877215
saving model with acc 0.728


## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))